In [1]:
#=======================================================================
# xml-roberta MLM 테스트 예제
# - MASK 토큰 -> <mask> 임.
#=======================================================================

from transformers import AutoTokenizer, RobertaForMaskedLM
import torch
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', do_lower_case=False)
model = RobertaForMaskedLM.from_pretrained('xlm-roberta-base')

text = ['한국의 수도는 <mask>이다', '에펠탑은 <mask>에 있다', '충무공 이순신은 <mask>에 최고의 장수였다']
tokenized_input = tokenizer(text, max_length=128, truncation=True, padding='max_length', return_tensors='pt')

outputs = model(**tokenized_input)
logits = outputs.logits

mask_idx_list = []
for tokens in tokenized_input['input_ids'].tolist():
    token_str = [tokenizer.convert_ids_to_tokens(s) for s in tokens]
    
    # **위 token_str리스트에서 [MASK] 인덱스를 구함
    # => **해당 <mask> 안덱스 값 mask_idx 에서는 아래 출력하는데 사용됨
    mask_idx = token_str.index('<mask>')
    mask_idx_list.append(mask_idx)
    
for idx, mask_idx in enumerate(mask_idx_list):
    
    logits_pred=torch.argmax(F.softmax(logits[idx]), dim=1)
    mask_logits_idx = int(logits_pred[mask_idx])
    # [MASK]에 해당하는 token 구함
    mask_logits_token = tokenizer.convert_ids_to_tokens(mask_logits_idx)
    # 결과 출력 
    print('\n')
    print('*Input: {}'.format(text[idx]))
    print('*[MASK] : {} ({})'.format(mask_logits_token, mask_logits_idx))

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
F:\AnacondaEnv\daEnv\bong\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.




*Input: 한국의 수도는 <mask>이다
*[MASK] : ▁서울 (20899)


*Input: 에펠탑은 <mask>에 있다
*[MASK] : ▁프랑스 (119633)


*Input: 충무공 이순신은 <mask>에 최고의 장수였다
*[MASK] : ▁대한민국 (81774)
